In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abdulrahmankags","key":"f6d6a7aef61e9f142fdf58218ca91a2a"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c mercedes-benz-greener-manufacturing

  0% 0.00/158k [00:00<?, ?B/s]
100% 158k/158k [00:00<00:00, 60.6MB/s]
  0% 0.00/10.7k [00:00<?, ?B/s]
100% 10.7k/10.7k [00:00<00:00, 9.59MB/s]
  0% 0.00/175k [00:00<?, ?B/s]
100% 175k/175k [00:00<00:00, 56.5MB/s]


In [4]:
from zipfile import ZipFile
files = ['sample_submission.csv.zip','train.csv.zip','test.csv.zip']
for i in files:
  with ZipFile(i,'r') as zip:
    zip.extractall()
    print('Done')

Done
Done
Done


In [5]:
# Importing required libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
import xgboost as xgb
import pandas as pd 
from sklearn.linear_model import LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline 
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error

In [6]:
# Reading train data file

train_data = pd.read_csv('train.csv')
train_data

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [7]:
# Reading test data file

test_data = pd.read_csv('test.csv')
test_data

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [15]:
def function_1(input_data):
  X = pd.read_csv(input_data)

  #if np.ndim(X) == 1:
  #  test_d = pd.DataFrame([X],columns=test_data.columns)
  #elif np.ndim(X) == 2:
  #  test_d = pd.DataFrame(X,columns=test_data.columns)
  test_df = X

  #Stacking model specified
  class Stacking(BaseEstimator,TransformerMixin):
    def __init__(self, estimator):
      self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
      self.estimator.fit(X, y, **fit_params)
      return self
    def transform(self, X):
      X = check_array(X) # Converting into array format
      X_new = np.copy(X) # Taking copy of existing X data
      # Checking for classification model and if so, checking also the model has attribute 'predict_proba'
      if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
        X_new = np.hstack((self.estimator.predict_proba(X), X)) # Stacking predicted values along with dataset

      X_new = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_new)) # Stacking predicted values along with dataset
      return X_new

  # Converting all the categorical columns into Ordinal encoding

  train_df = train_data.copy()
  for c in train_df.columns:
    if train_df[c].dtype == 'object': # Checking for columns with 'object' datatype
      mapper = lambda x:sum([ord(digit) for digit in x])
      train_df[c] = train_df[c].apply(mapper) # Transforming columns of train data
      test_df[c] = test_df[c].apply(mapper) # Transforming columns of test data
  
  Y_train = train_df['y']
  X_train = train_df.drop(['y'],axis=1)
  X_test = test_df ################################# Ordinal encoded normal features

  stacked_pipeline = make_pipeline(
  Stacking(estimator=LassoLarsCV(normalize=True)),
  Stacking(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)),
  LassoLarsCV())

  stacked_pipeline.fit(X_train,Y_train)
  pred_normal = stacked_pipeline.predict(X_test)
 
  # Decomposed features

  n_comp = 12

  # tSVD 
  tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
  tsvd_results_train = tsvd.fit_transform(train_df.drop(["y"], axis=1))
  tsvd_results_test = tsvd.transform(test_df)

  # PCA
  pca = PCA(n_components=n_comp, random_state=420)
  pca2_results_train = pca.fit_transform(train_df.drop(["y"], axis=1))
  pca2_results_test = pca.transform(test_df)

  # ICA https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html
  ica = FastICA(n_components=n_comp, random_state=420)
  ica2_results_train = ica.fit_transform(train_df.drop(["y"], axis=1))
  ica2_results_test = ica.transform(test_df)

  # GRP https://scikit-learn.org/stable/modules/generated/sklearn.random_projection.GaussianRandomProjection.html
  grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
  grp_results_train = grp.fit_transform(train_df.drop(["y"], axis=1))
  grp_results_test = grp.transform(test_df)

  # SRP https://scikit-learn.org/stable/modules/generated/sklearn.random_projection.SparseRandomProjection.html
  srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
  srp_results_train = srp.fit_transform(train_df.drop(["y"], axis=1))
  srp_results_test = srp.transform(test_df)

  # Including all the decomposed features into train and test dataset

  for i in range(n_comp):
    train_df['pca_' + str(i)] = pca2_results_train[:, i] # Including pca features
    test_df['pca_' + str(i)] = pca2_results_test[:, i]

    train_df['ica_' + str(i)] = ica2_results_train[:, i] # Including ica features
    test_df['ica_' + str(i)] = ica2_results_test[:, i]

    train_df['tsvd_' + str(i)] = tsvd_results_train[:, i] # Including svd features
    test_df['tsvd_' + str(i)] = tsvd_results_test[:, i]

    train_df['grp_' + str(i)] = grp_results_train[:, i] # Including grp features
    test_df['grp_' + str(i)] = grp_results_test[:, i]

    train_df['srp_' + str(i)] = srp_results_train[:, i] # Including srp features
    test_df['srp_' + str(i)] = srp_results_test[:, i]

  X_train_decom = train_df.drop(['y'],axis=1)
  X_test_decom = test_df
  y_mean = np.mean(Y_train)
  id_test = test_df['ID'].values
  

  # https://xgboost.readthedocs.io/en/latest/parameter.html
  # Reference : https://www.kaggle.com/gowtamsingulur/stacking-greener-benz

  xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, 
    'silent': 1
  }

  # Converting to Dmatrix to make XGboost work faster and efficient
  dtrain = xgb.DMatrix(X_train_decom,Y_train) #https://xgboost.readthedocs.io/en/latest/python/python_intro.html
  dtest = xgb.DMatrix(X_test_decom) # Here we are using decomposed features

  model_xgb = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=1250)
  pred_decom = model_xgb.predict(dtest)

  Y_final = pred_decom*0.75 + pred_normal*0.25 # Taking weights on xgboost result and stacking result respectively
  return Y_final

In [9]:
Y = function_1('test.csv') # Computing for full set of points
Y

[10:24:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([ 81.06829025,  99.83590356,  80.3205701 , ...,  92.68605458,
       110.30346141,  92.16208046])

In [10]:
final = pd.DataFrame()
final['ID'] = test_data['ID']
final['y'] = Y
final.to_csv('Final_predictions_.csv', index=False)
final

,ID,y
0,1,81.068290
1,2,99.835904
2,3,80.320570
3,4,79.112151
4,5,113.631200
...,...,...
4204,8410,102.220053
4205,8411,92.174323
4206,8413,92.686055
4207,8414,110.303461


In [11]:
from google.colab import files
files.upload()

Saving Sample2.csv to Sample2.csv
Saving Sample1.csv to Sample1.csv
Saving Sample3.csv to Sample3.csv


{'Sample1.csv': b'ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X22

In [12]:
check_sample = function_1('Sample3.csv') # Checking one sample point
check_sample

[10:28:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([99.85451244])

In [13]:
check_sample2 = function_1('Sample1.csv') # Checking one sample point
check_sample2

[10:29:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([81.06834194])

In [16]:
def function2(X,Y_test):

  if np.ndim(X) == 1:
    test_d = pd.DataFrame([X],columns=test_data.columns)
  elif np.ndim(X) == 2:
    test_d = pd.DataFrame(X,columns=test_data.columns)
  test_df = test_d

  #Stacking model specified
  class Stacking(BaseEstimator,TransformerMixin):
    def __init__(self, estimator):
      self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
      self.estimator.fit(X, y, **fit_params)
      return self
    def transform(self, X):
      X = check_array(X) # Converting into array format
      X_new = np.copy(X) # Taking copy of existing X data
      # Checking for classification model and if so, checking also the model has attribute 'predict_proba'
      if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
        X_new = np.hstack((self.estimator.predict_proba(X), X)) # Stacking predicted values along with dataset

      X_new = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_new)) # Stacking predicted values along with dataset
      return X_new

  # Converting all the categorical columns into Ordinal encoding
  train_df = train_data.copy()
  for c in train_df.columns:
    if train_df[c].dtype == 'object': # Checking for columns with 'object' datatype
      mapper = lambda x:sum([ord(digit) for digit in x])
      train_df[c] = train_df[c].apply(mapper) # Transforming columns of train data
      test_df[c] = test_df[c].apply(mapper) # Transforming columns of test data
  
  Y_train = train_df['y']
  X_train = train_df.drop(['y'],axis=1)
  X_test = test_df ################################# Ordinal encoded normal features

  stacked_pipeline = make_pipeline(
  Stacking(estimator=LassoLarsCV(normalize=True)),
  Stacking(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)),
  LassoLarsCV())

  stacked_pipeline.fit(X_train,Y_train)
  pred_normal = stacked_pipeline.predict(X_test)
 
  # Decomposed features

  n_comp = 12

  # tSVD 
  tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
  tsvd_results_train = tsvd.fit_transform(train_df.drop(["y"], axis=1))
  tsvd_results_test = tsvd.transform(test_df)

  # PCA
  pca = PCA(n_components=n_comp, random_state=420)
  pca2_results_train = pca.fit_transform(train_df.drop(["y"], axis=1))
  pca2_results_test = pca.transform(test_df)

  # ICA https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html
  ica = FastICA(n_components=n_comp, random_state=420)
  ica2_results_train = ica.fit_transform(train_df.drop(["y"], axis=1))
  ica2_results_test = ica.transform(test_df)

  # GRP https://scikit-learn.org/stable/modules/generated/sklearn.random_projection.GaussianRandomProjection.html
  grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
  grp_results_train = grp.fit_transform(train_df.drop(["y"], axis=1))
  grp_results_test = grp.transform(test_df)

  # SRP https://scikit-learn.org/stable/modules/generated/sklearn.random_projection.SparseRandomProjection.html
  srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
  srp_results_train = srp.fit_transform(train_df.drop(["y"], axis=1))
  srp_results_test = srp.transform(test_df)

  # Including all the decomposed features into train and test dataset

  for i in range(n_comp):
    train_df['pca_' + str(i)] = pca2_results_train[:, i] # Including pca features
    test_df['pca_' + str(i)] = pca2_results_test[:, i]

    train_df['ica_' + str(i)] = ica2_results_train[:, i] # Including ica features
    test_df['ica_' + str(i)] = ica2_results_test[:, i]

    train_df['tsvd_' + str(i)] = tsvd_results_train[:, i] # Including svd features
    test_df['tsvd_' + str(i)] = tsvd_results_test[:, i]

    train_df['grp_' + str(i)] = grp_results_train[:, i] # Including grp features
    test_df['grp_' + str(i)] = grp_results_test[:, i]

    train_df['srp_' + str(i)] = srp_results_train[:, i] # Including srp features
    test_df['srp_' + str(i)] = srp_results_test[:, i]

  X_train_decom = train_df.drop(['y'],axis=1)
  X_test_decom = test_df
  y_mean = np.mean(Y_train)
  id_test = test_df['ID'].values

  # https://xgboost.readthedocs.io/en/latest/parameter.html
  # Reference : https://www.kaggle.com/gowtamsingulur/stacking-greener-benz

  xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, 
    'silent': 1
  }

  # Converting to Dmatrix to make XGboost work faster and efficient
  dtrain = xgb.DMatrix(X_train_decom,Y_train) #https://xgboost.readthedocs.io/en/latest/python/python_intro.html
  dtest = xgb.DMatrix(X_test_decom) # Here we are using decomposed features

  model_xgb = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=1250)
  pred_decom = model_xgb.predict(dtest)

  Y_final = pred_decom*0.75 + pred_normal*0.25 # Taking weights on xgboost result and stacking result respectively

  # Computing performance value

  R_square = r2_score(Y_test,Y_final)
  MSLE = mean_squared_log_error(Y_test,Y_final)

  return R_square,MSLE

In [17]:
Y_data = train_data['y']

In [18]:
R_square,MSLE = function2(train_data.drop(['y'],axis=1),Y_data)

[10:36:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [19]:
print('R² value of data given in function2:',R_square) # Performance calculated given set of points and targets
print('MSLE value of data given in function2:',MSLE)

R² value of data given in function2: 0.6563700407974008
MSLE value of data given in function2: 0.004394474900501425


In [20]:
Y_data[[0]]

0    130.81
Name: y, dtype: float64

In [21]:
R_square,MSLE = function2(train_data.drop(['y'],axis=1).values[0],Y_data[[0]]) # Performance calculated given single point and target

[10:38:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [22]:
print('R² value of data given in function2:',R_square) # For single point R² can't be calculated
print('MSLE value of data given in function2:',MSLE)

R² value of data given in function2: nan
MSLE value of data given in function2: 0.036646805332088896
